In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 24.8 MB/s            
     |████████████████████████████████| 880 kB 64.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.6 MB 56.0 MB/s            
     |████████████████████████████████| 67 kB 8.3 MB/s             
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895254 sha256=43b63edc28a16633420946ce895391ed7c64b957b7d12c151203d7e9d4e640b1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built sacremoses
     |████████████████████████████████| 346 kB 23.5 MB/s            
     |████████████████████████████████| 133 kB 71.9 MB/s            
     |████████████████████████████████| 211 kB 69.8 MB/s            
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.4.0
    Uninstalling fsspec-2021.4.0:
      Successfully uninstalled fsspec-2021.4.0
ERROR: pip

In [2]:
import datasets
import transformers

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Load data set & Preprocessing

+ Won't remove stop word here

In [5]:
#remove punctuation, remove marks, remove urls, lower case,remove numbers, lowercase
#remove ascii chatachters
def removePuncStr(s):
    for c in string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§':
        s = s.replace(c, ' ').strip()
    return s

def removePunc(array):
    return [removePuncStr(str_) for str_ in array]

#remove numbers
def removeNumbersStr(s):
    for d in range(10):
        s = s.replace(str(d), ' ')
    return s

def removeNumbers(array):
    return [removeNumbersStr(str_) for str_ in array]

def lemmatize(text_array):
    WNlemmatizer = WordNetLemmatizer()
    lemmatized_text = []
    for h in text_array:
        words = word_tokenize(h)
        h2 = ''
        for w in words:
            h2 = h2 + ' ' + WNlemmatizer.lemmatize(w)
        lemmatized_text.append(h2)
    return lemmatized_text

def removeSpacesStr(s):
    spaces = ['\t', '\r', '\n']
    for space in spaces:
        s = s.replace(space, ' ')
        
def removeSpaces(array):
    return [removeNumbersStr(sent) for sent in array]

def Lowercase(array):
    return [str(sent).lower() for sent in array]

def removeUrl(text_array):
    pattern = r"https://.+\S*"
    removed = []
    for s in text_array:
        s = re.sub(pattern, ' ', s)
        removed.append(s)
    return removed
def removeAscii(text_array):
    return [re.sub(r'[^\x00-\x7f]', '', s) for s in text_array]

In [6]:
def clean(text_array):
    text_array = removeUrl(text_array)
    text_array = removePunc(text_array)
    text_array = removeNumbers(text_array)
    text_array = removeSpaces(text_array)
    text_array = Lowercase(text_array)
    text_array = lemmatize(text_array)
    text_array = removeAscii(text_array)
    return text_array

In [7]:
label_mapper = {
    'Neutral' : 0,
    'Positive' : 1,
    'Extremely Positive' : 2,
    'Negative' : 3,
    'Extremely Negative' : 4
}

In [8]:
# prepare the data for initializing a dataset for Bert
def makeDataset(dataset):
    dataset = dataset[['OriginalTweet', 'Sentiment']]
    text_array = dataset['OriginalTweet']
    text_array = clean(text_array)
    dataset['Text'] = text_array
    dataset = dataset.drop(columns=['OriginalTweet'])
    dataset = dataset.rename(columns = {'Sentiment': 'Labels'})
    dataset['Labels'] = dataset['Labels'].map(label_mapper)
    return dataset

In [9]:
train_set = pd.read_csv('Corona_NLP_train.csv', encoding='ISO-8859-1')
test_set = pd.read_csv('Corona_NLP_test.csv', encoding='ISO-8859-1')

train_set = makeDataset(train_set)
test_set = makeDataset(test_set)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
train_set.head()

,Labels,Text
0,0,menyrbie phil gahan chrisitv
1,1,advice talk to your neighbour family to excha...
2,1,coronavirus australia woolworth to give elder...
3,1,my food stock is not the only one which is em...
4,4,me ready to go at supermarket during the covi...


In [11]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_set)
test_data = Dataset.from_pandas(test_set)

In [12]:
label_id = {
    'Neutral' : 0,
    'Positive' : 1,
    'Extremely Positive' : 2,
    'Negative' : 3,
    'Extremely Negative' : 4
}

In [13]:
id2label = {id_:name for name, id_ in label_id.items()}
id2label

{0: 'Neutral',
 1: 'Positive',
 2: 'Extremely Positive',
 3: 'Negative',
 4: 'Extremely Negative'}

In [14]:
#splits train and validation set
#use train_test_split in Datasets module

splitted = train_data.train_test_split(test_size=0.2, shuffle=True, seed=222)
train_data, val_data = splitted['train'], splitted['test']

from datasets import DatasetDict
datasets = DatasetDict({'train' : train_data, 'val' : val_data, 'test' : test_data})


In [15]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Labels', 'Text'],
        num_rows: 32925
    })
    val: Dataset({
        features: ['Labels', 'Text'],
        num_rows: 8232
    })
    test: Dataset({
        features: ['Labels', 'Text'],
        num_rows: 3798
    })
})

In [16]:
## Load Bert Model and Tokenizer
from transformers import BertForSequenceClassification, BertTokenizer

bertModel = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_id), id2label=id2label, label2id=label_id)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [17]:
## find max sequence length
max_length = 0
max_text = ''

for text in train_set['Text']:
    if(len(text.split(' ')) > max_length):
        max_length = len(text.split(' '))
        max_text = text
print(max_length)
print(max_text)

66
 i want to go shopping i want to walk w o the thought i might get the covid virus i want a real hug i want to go eat at a dine in restaurant ohhh to be free w o the thought of that darn virus i want to go shopping for shoe i don t need just want to go shopping no more online


In [18]:
%%time

max_sequence_length = 128

def tokenization(dataset):
    
    args = ((dataset['Text'], ))
    result = tokenizer(*args, padding='max_length', max_length=max_sequence_length, truncation=True)
    result['label'] = [label for label in dataset['Labels']]
    
    return result

datasets = datasets.map(tokenization, batched=True, desc='Tokenization')

Tokenization:   0%|          | 0/33 [00:00<?, ?ba/s]

Tokenization:   0%|          | 0/9 [00:00<?, ?ba/s]

Tokenization:   0%|          | 0/4 [00:00<?, ?ba/s]

CPU times: user 1min 5s, sys: 1.05 s, total: 1min 6s
Wall time: 1min 5s


In [19]:
datasets['train']

Dataset({
    features: ['Labels', 'Text', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 32925
})

In [20]:
for i in range(len(datasets['train'])):
    if(i<=3):
        print('keys: ', datasets['train'][i].keys())
        print('text: ', datasets['train'][i]['Text'])
        print(len(datasets['train'][i]['input_ids']))
        print('tokenized sequence :', tokenizer.decode(datasets['train'][i]['input_ids']))
        print('end-----------------------------------------------------------------------')

keys:  dict_keys(['Labels', 'Text', 'input_ids', 'token_type_ids', 'attention_mask', 'label'])
text:   overheard at the grocery store i like frozen food better than fresh anyways  boyyyy do i have an ex for you girl coronavirus
128
tokenized sequence : [CLS] overheard at the grocery store i like frozen food better than fresh anyways boyyyy do i have an ex for you girl coronavirus [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
end--------------------

In [21]:
## Training

from datasets import load_metric

metric_name = 'accuracy'
metric = load_metric(metric_name)

def compute_metrics(returned):
    y_pred = returned.predictions[0] if isinstance(returned.predictions, tuple) else returned.predictions
    y_pred = np.argmax(y_preds, axis=1)
    
    return {"accuracy" : (y_pred == returned.label_ids).astype(np.float32).mean().item()}

In [22]:
## Set Training Arguments

from transformers import Trainer, TrainingArguments

batch_size = 32

training_args = TrainingArguments(
'Corona_NLP_Bert',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate= 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to='none'
)

In [23]:
## Initialize Trainer
debug = False

trainer = Trainer(
    model = bertModel,
    args = training_args,
    train_dataset=datasets['train'].select(range(10)) if debug else datasets['train'],
    eval_dataset = datasets['val'].select(range(10)) if debug else datasets['val'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
## Train
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text, Labels. If Text, Labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32925
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4116


In [ ]:
trainer.evaluate()

In [ ]:
# test_set = datasets['test']
# test_set = test_set.remove_columns("Labels")
# y_pred = trainer.predict(test_set, )